In [7]:
from tensorflow.keras.models import Model # Импортируем модели keras: Model
from tensorflow.keras.layers import * # Импортируем все слои keras
from tensorflow.keras import backend as K # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam # Импортируем оптимизатор Adam
from tensorflow.keras import utils # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
import tensorflow as tf
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями
import numpy as np # Импортируем библиотеку numpy
from sklearn.model_selection import train_test_split
import time
import random
import shutil
import os # Импортируем библиотеку os для раоты с фаловой системой
from PIL import Image # импортируем модель Image для работы с изображениями
import seaborn as sns


import numba as nb
import cv2 as cv
import json
from PIL import Image, ImageDraw
from numpy.lib.stride_tricks import as_strided
sns.set_style('darkgrid')

In [8]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
directory = '/content/drive/My Drive/segmentation/' # Указываем путь к обучающей выборке с оригинальными изображения

# Загружаем и обрабатываем данные

In [50]:
with open(directory + "via_POLIGON_data.json", "r") as write_file:
    data = json.load(write_file)

In [51]:
images = []
for filename in data.values():
    temp_img = np.array(Image.open(directory + 'orig/' + 'дог' + str(int(filename['filename'][0]) + 1) + '.jpg'))
    temp_img = cv.resize(temp_img, (768, 512), interpolation = cv.INTER_NEAREST)
    images.append(temp_img)


images = np.array(images)

In [52]:
all_points = []

for image_data in data.values():
  point = image_data['regions']['0']['shape_attributes']
  four_coords = tuple(zip(point['all_points_x'], point['all_points_y']))
  all_points.append(four_coords)

all_points = np.array(all_points)[:, 0:4, :].astype(np.float32)
all_points[:, :, 0] = all_points[:, :, 0]/(images.shape[2] * 2)
all_points[:, :, 1] = all_points[:, :, 1]/(images.shape[1] * 2)
all_points = all_points.reshape(-1, 8)

In [54]:
x_train, x_test, y_train, y_test = train_test_split(images, all_points, test_size = 0.2)

# Создаем Нейросеть

In [55]:
def reslayer(*args, x, n = 5, **kwargs):
  y = Conv2D(padding = 'same', activation = 'relu', *args, **kwargs)(x)
  for i in range(n):
    y = Conv2D(padding = 'same', activation = 'relu', *args, **kwargs)(y)
  return y

In [61]:
# input_layer = Input((x_train.shape[1], x_train.shape[2], x_train.shape[3]))
# Conv_1 = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_layer)

# Conv_2 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_1)
# Conv_2 = reslayer(64, (3, 3), x = Conv_2)

# Conv_3 = Conv2D(96, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_2)
# Conv_3 = reslayer(96, (3, 3), x = Conv_3)

# Conv_4 = Conv2D(128, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_3)
# Conv_4 = reslayer(128, (3, 3), x = Conv_4)

# Conv_5 = Conv2D(160, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_4)
# Conv_5 = reslayer(160, (3, 3), x = Conv_5)

# Conv_6 = Conv2D(180, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_5)
# Conv_6 = reslayer(180, (3, 3), x = Conv_6)

# Conv_7 = Conv2D(200, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_6)
# Conv_7 = reslayer(200, (3, 3), x = Conv_7)

# Conv_8 = Conv2D(256, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_7)
# Conv_8 = reslayer(256, (3, 3), x = Conv_8)

# Conv_9 = Conv2D(512, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_8)
# Conv_9 = reslayer(512, (3, 3), x = Conv_9)

# flatten_layer = Flatten()(Conv_9)
# dense1 = Dense(512, activation = 'relu')(flatten_layer)
# dense1 = Dropout(0.2)(dense1)
# dense1 = BatchNormalization()(dense1)
# dense2 = Dense(256, activation = 'relu')(dense1)
# dense2 = Dropout(0.2)(dense2)
# dense2 = BatchNormalization()(dense2)

# output_layer = Dense(8, activation = 'sigmoid')(dense2)

# model = Model(input_layer, output_layer)

# model.compile(optimizer=Adam(lr = 0.0002), loss='binary_crossentropy', metrics=['mae'])

In [59]:
input_layer = Input((x_train.shape[1], x_train.shape[2], x_train.shape[3]))
Conv_1 = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_layer)

Conv_2 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_1)
Conv_2v1 = reslayer(96, (6, 3), strides = (2, 2), x = Conv_2)
Conv_2v2 = reslayer(96, (3, 6), strides = (2, 2), x = Conv_2)
Conv_2 = Concatenate()([Conv_2v1, Conv_2v2])

flatten_layer = Flatten()(Conv_2)
dense1 = Dense(512, activation = 'relu')(flatten_layer)
dense1 = Dropout(0.2)(dense1)
dense1 = BatchNormalization()(dense1)
dense2 = Dense(256, activation = 'relu')(dense1)
dense2 = Dropout(0.2)(dense2)
dense2 = BatchNormalization()(dense2)

output_layer = Dense(8, activation = 'sigmoid')(dense2)

model = Model(input_layer, output_layer)

model.compile(optimizer=Adam(lr = 0.0002), loss='binary_crossentropy', metrics=['mae'])

In [62]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 512, 768, 3)]     0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 512, 768, 32)      896       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 256, 384, 64)      18496     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 256, 384, 64)      36928     
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 256, 384, 64)      36928     
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 256, 384, 64)      36928     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 256, 384, 64)      3692

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
cb = ModelCheckpoint(
    '/content/drive/My Drive/segmentation/weightsRegressor/',
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min",
    save_freq="epoch",
    options=None
)

history = model.fit(x_train/255, y_train, epochs=150, batch_size=4, validation_data = (x_test/255, y_test), callbacks = [cb]) #  Обучаем модель на выборке по трем классам на полноразмерных изображениях

Epoch 1/150
21/21 [==============================] - 12s 427ms/step - loss: 0.6934 - mae: 0.4592 - val_loss: 0.6797 - val_mae: 0.4446

Epoch 00001: val_loss improved from inf to 0.67968, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 2/150
21/21 [==============================] - 8s 355ms/step - loss: 0.6762 - mae: 0.4488 - val_loss: 0.6633 - val_mae: 0.4355

Epoch 00002: val_loss improved from 0.67968 to 0.66328, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 3/150
21/21 [==============================] - 8s 365ms/step - loss: 0.6581 - mae: 0.4391 - val_loss: 0.6422 - val_mae: 0.4235

Epoch 00003: val_loss improved from 0.66328 to 0.64220, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 4/150
21/21 [==============================] - 8s 367ms/step - loss: 0.6351 - mae: 0.4278 - val_loss: 0.6148 - val_mae: 0.4075

Epoch 00004: val_loss improved from 0.64220 to 0.61479, saving model to /content/drive

In [ ]:
#model.save(directory + '/weightsRegressor/regressor_model_v2.h5')

In [ ]:
def gather_four_coords(points):
  results = [(round(points[i] * x_train.shape[2]), round(points[i + 1] * x_train.shape[1])) for i in range(0, len(points), 2)] + [(round(points[0] * x_train.shape[2]), round(points[1] * x_train.shape[1]))]
  return results

predicted_coords = model.predict(x_test/255, batch_size = 2, verbose = 0)
for i, image in enumerate(x_test):
  image = Image.fromarray(image)
  points = gather_four_coords(predicted_coords[i])
  draw = ImageDraw.Draw(image)
  draw.line(points, width = 5, fill="yellow")
  image.save(directory + f"regressor_predicted/basic_results_total/{i}.jpg")

Итоги:
* Нужно обучать на большом количестве эпох
* resnet не даёт более хорошие результаты
* Dropout в свёрточных слоях ухудшает результаты
* binary_crossentropy быстрее обучается